In [ ]:
!pip install azure-identity
!pip install langchain
!pip install openai
!pip install azure-search-documents

In [12]:
import os
import openai
from azure.identity import DefaultAzureCredential
from azure.search.documents import SearchClient
from azure.search.documents.models import QueryType

In [ ]:

!pip3 install python-dotenv
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

# If your enviornment variable giving you previous values,
# then delete existing one And know set new one
# del os.environ['AZURE_SEARCH_SERVICE']

# Replace these with your own values, either in environment variables or directly here
AZURE_STORAGE_ACCOUNT = os.environ.get("AZURE_STORAGE_ACCOUNT")  # This is only in use if you want to store your data in azure blob storage
AZURE_STORAGE_CONTAINER = os.environ.get("AZURE_STORAGE_CONTAINER")

AZURE_SEARCH_SERVICE = os.environ.get("AZURE_SEARCH_SERVICE")
AZURE_SEARCH_INDEX = os.environ.get("AZURE_SEARCH_INDEX")
AZURE_SEARCH_KEY = os.environ.get("AZURE_SEARCH_KEY")

AZURE_OPENAI_SERVICE = os.environ.get("AZURE_OPENAI_SERVICE")
AZURE_OPENAI_GPT_DEPLOYMENT = os.environ.get("AZURE_OPENAI_GPT_DEPLOYMENT")
AZURE_OPENAI_CHATGPT_DEPLOYMENT = os.environ.get("AZURE_OPENAI_CHATGPT_DEPLOYMENT")
AZURE_OPENAI_CHATGPT_MODEL = os.environ.get("AZURE_OPENAI_CHATGPT_MODEL")

OPENAI_API_KEY= os.environ.get("OPENAI_API_KEY")
# AZURE_TENANT_ID = os.environ.get("AZURE_TENANT_ID")
# AZURE_CLIENT_SECRET = os.environ.get["AZURE_CLIENT_SECRET"]
# AZURE_CLIENT_ID = os.environ.get["AZURE_CLIENT_ID"]

KB_FIELDS_CONTENT = os.environ.get("KB_FIELDS_CONTENT")
KB_FIELDS_CATEGORY = os.environ.get("KB_FIELDS_CATEGORY")
KB_FIELDS_SOURCEPAGE = os.environ.get("KB_FIELDS_SOURCEPAGE")


In [41]:
# Use the current user identity to authenticate with Azure OpenAI, Cognitive Search and Blob Storage (no secrets needed,
# just use 'az login' locally, and managed identity when deployed on Azure). If you need to use keys, use separate AzureKeyCredential instances with the
# keys for each service
azure_credential = DefaultAzureCredential()


# Used by the OpenAI SDK
openai.api_type = "azure"
openai.api_base = f"https://{AZURE_OPENAI_SERVICE}.openai.azure.com"
openai.api_version = "2023-05-15"


# Comment these two lines out if using keys, set your API key in the OPENAI_API_KEY environment variable instead
# openai.api_type = "azure_ad"
# openai.api_key = azure_credential.get_token("https://cognitiveservices.azure.com/.default").token
openai.api_key=OPENAI_API_KEY

# Set up clients for Cognitive Search and Storage
search_client = SearchClient(
    endpoint=f"https://{AZURE_SEARCH_SERVICE}.search.windows.net",
    index_name=AZURE_SEARCH_INDEX,
    credential=AZURE_SEARCH_KEY)


In [45]:
# Chat roles
SYSTEM = "system"
USER = "user"
ASSISTANT = "assistant"

# previous chat
system_message_chat_conversation = """Assistant helps the company employees with their healthcare plan questions, and questions about the employee handbook. Be brief in your answers.
Answer ONLY with the facts listed in the list of sources below. If there isn't enough information below, say you don't know. Do not generate answers that don't use the sources below. If asking a clarifying question to the user would help, ask the question.
Each source has a name followed by colon and the actual information, always include the source name for each fact you use in the response. Use square brackets to reference the source, e.g. [info1.txt]. Don't combine sources, list each source separately, e.g. [info1.txt][info2.pdf].
"""
chat_conversations = [{"role" : SYSTEM, "content" : system_message_chat_conversation}]

summary_prompt_template = """Below is a summary of the conversation so far, and a new question asked by the user that needs to be answered by searching in a knowledge base. Generate a search query based on the conversation and the new question. Source names are not good search terms to include in the search query.

Summary:
{summary}

Question:
{question}

Search query:
"""

In [56]:
# Execute this cell multiple times updating user_input to accumulate chat history
user_input = "Does my plan cover annual eye exams? Any type"

# Exclude category, to simulate scenarios where there's a set of docs you can't see
exclude_category = None

# To check if there is any chat history available
if len(chat_conversations) > 1:
    # If there is previous chat then provide the summary
    query_completion = openai.Completion.create(
        engine=AZURE_OPENAI_GPT_DEPLOYMENT,
        prompt=summary_prompt_template.format(summary=str(chat_conversations), question=user_input),
        temperature=0.7,
        max_tokens=32,
        stop=["\n"])
    search = query_completion.choices[0].text
else:
  # set the user input to search if there is no chat history
    search = user_input


In [ ]:
# Alternatively simply use search_client.search(q, top=3) if not using semantic search
print("Searching:", search)
print("-------------------")
filter = "category ne '{}'".format(exclude_category.replace("'", "''")) if exclude_category else None
r = search_client.search(search,
                         filter=filter,
                        #  query_type=QueryType.SEMANTIC,
                         query_language="en-us",
                         query_speller="lexicon",
                         semantic_configuration_name="default",
                         top=3)



In [ ]:
# Assuming KB_FIELDS_SOURCEPAGE and KB_FIELDS_CONTENT are the actual field names you're using
results = []

for doc in r:
    source_page = doc.get(KB_FIELDS_SOURCEPAGE, "")
    content = doc.get(KB_FIELDS_CONTENT, "").replace("\n", "").replace("\r", "")
    result = f"{source_page}: {content}"
    results.append(result)

content = "\n".join(results)
user_content = user_input + " \nSOURCES:\n" + content
print(user_content)  # Print the user_content, not 'result'

In [61]:
user_content = user_input + " \nSOURCES:\n" + content

chat_conversations.append({"role": USER, "content": user_content })

chat_completion = openai.ChatCompletion.create(
    deployment_id=AZURE_OPENAI_CHATGPT_DEPLOYMENT,
    model=AZURE_OPENAI_CHATGPT_MODEL,
    messages=chat_conversations,
    temperature=0.7,
    max_tokens=1024,
    n=1)
chat_content = chat_completion.choices[0].message.content
'''
reset user content to avoid sources in conversation history
add source as a single shot in query conversation
'''
chat_conversations[-1]["content"] = user_input
chat_conversations.append({"role":ASSISTANT, "content": chat_content})

print("\n-------------------\n")
[print(conversation) for conversation in chat_conversations]


-------------------

{'role': 'system', 'content': "Assistant helps the company employees with their healthcare plan questions, and questions about the employee handbook. Be brief in your answers.\nAnswer ONLY with the facts listed in the list of sources below. If there isn't enough information below, say you don't know. Do not generate answers that don't use the sources below. If asking a clarifying question to the user would help, ask the question.\nEach source has a name followed by colon and the actual information, always include the source name for each fact you use in the response. Use square brackets to reference the source, e.g. [info1.txt]. Don't combine sources, list each source separately, e.g. [info1.txt][info2.pdf].\n"}
{'role': 'user', 'content': 'Does my plan cover annual eye exams? \nSOURCES:\n'}
{'role': 'user', 'content': 'Does my plan cover annual eye exams?'}
{'role': 'assistant', 'content': "I'm sorry, I need more information. Can you please tell me what type of h

[None, None, None, None, None, None]